In [ ]:
# Initialisation du client Binance
client = Client()

# Paramètres de l'extraction
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1DAY
start_date = '1 Jan, 2023'
end_date = '31 Dec, 2023'

# Extraire les données
df = get_historical_data(symbol, interval, start_date, end_date)

# Sauvegarder les données dans un fichier CSV
output_file = "BTCUSDT_daily_2023.csv"
df.to_csv(output_file, index=False)
print(f"Données sauvegardées dans {output_file}")

In [ ]:
# Charger les données depuis le fichier CSV
data_file = "BTCUSDT_daily_2023.csv"
df = pd.read_csv(data_file)

# Convertir la colonne 'timestamp' en format datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Vérifier les données
print(df.head())
print(df.info())

In [ ]:
# Graphique simple des prix de clôture
plt.figure(figsize=(12, 6))
plt.plot(df['timestamp'], df['close'], label='Prix de clôture')
plt.title("Prix de clôture BTC/USDT (2023)")
plt.xlabel("Date")
plt.ylabel("Prix (USDT)")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Checking for any missing (NaN) values in the dataset.
df.isna().sum()

In [ ]:
final_data = add_indicators(df)
final_data

In [ ]:
scaled_data = scaling_data(final_data)
scaled_data

In [ ]:
# Setting up parameters for a prediction model. Here, the goal is to predict the closing price 60 days in the future using the closing price as the regressor.
prediction_time = 60 # We want to predict the close price 60 days Later 
regressor = ['close'] # We choose One regressor which is the colse price itself
price, target  = data_preprocess(scaled_data, regressor, prediction_time) # Data Processing for the model

In [ ]:
# Applying linear regression to the preprocessed data.
regression_matrix, future_price, simple_r2 = apply_linear_regression(scaled_data, prediction_time, price, target, regressor)
# Visualizing the model's performance.
visualize_model(regression_matrix, scaled_data)

In [ ]:
visualize_model(regression_matrix, scaled_data, zoom = [600, 700])

In [ ]:
# Displaying the R-squared value of the linear regression model.
simple_r2

In [ ]:
# Visualizing future price predictions made by the model.
visualize_future(scaled_data, future_price)

In [ ]:
# Creating a correlation matrix from the final_data.
corr_matrix = final_data.drop('timestamp', axis=1).corr()
# A heatmap is then plotted using seaborn to visually represent the correlations.
sns.heatmap(corr_matrix)
plt.show

In [ ]:
# Extracting the correlation values of all features with respect to the 'close' column from the correlation matrix.
corr_matrix['close']

In [ ]:
# Updating the list of regressors to include 'close', 'ATR', and 'EMA' for the data preprocessing step.
regressor = ['close', 'ATR', 'EMA']
price, target  = data_preprocess(scaled_data, regressor, prediction_time) # Data Processing for the model

In [ ]:
# Applying linear regression with the updated set of regressors.
regression_matrix, future_price, mult_r2 = apply_linear_regression(scaled_data, prediction_time, price, target, regressor)

In [ ]:
# Visualizing the performance of the linear regression model.
visualize_model(regression_matrix, scaled_data, zoom=[600,700])

In [ ]:
# Displaying the R-squared values for both the multivariate model ('mult_r2') and the previous simple model ('simple_r2').
mult_r2, simple_r2

In [ ]:
visualize_future(scaled_data, future_price)

In [ ]:
# Creating a parameter grid for hyperparameter tuning. This grid includes a range of values for 'C' and 'gamma', which are parameters used in models like SVMs.
param_grid = {'C': [10**i for i in range(2, 11)], 
              'gamma' : [10**(-i) for i in range(2, 11)]} # Creat a dictionnary of parameters

In [ ]:
# Splitting the price and target data into training and test sets, with 70% of data as training set.
price_train, _, target_train, _ = train_test_split(price, target, test_size = 0.3)
# The 'cross_validation_parameter' function is then used to find the best 'C' and 'gamma' values for a model, likely through a grid search approach.
best_C, best_gamma = cross_validation_parameter(param_grid, price_train, target_train)

In [ ]:
print(f'The best values are {best_C} for C and {best_gamma} for Gamma')

In [ ]:
# Applying a Support Vector Regression (SVR) model with the best 'C' and 'gamma' values found. 
svr_matrix, future_price, accuracy_score = apply_svr(scaled_data, prediction_time, price, target, regressor, best_C, best_gamma)

In [ ]:
# Visualizing the SVR model's performance.
visualize_model(svr_matrix, scaled_data, zoom = [600, 700])

In [ ]:
# Displaying the accuracy score of the SVR model.
accuracy_score

In [ ]:
visualize_future(scaled_data, future_price)

In [ ]:
# Setting the sequence length for time series prediction.
sequence_length = 20 

In [ ]:
# Creating sequences from the scaled data using the specified sequence length.
X, y = create_sequences(scaled_data, sequence_length) # Prediction based on 20 periods (20 days or 20 hours) 
# The 'lstm_model' function trains the LSTM on these sequences and makes predictions.
y_train, y_test, predicted_values = lstm_model(X, y)

In [ ]:
visualize_RNN_prediction(y_train, y_test, predicted_values)

In [ ]:
data_file = "BTCUSDT_hourly_2023.csv"
hour_data = pd.read_csv(data_file)

# Convertir la colonne 'timestamp' en format datetime
hour_data['timestamp'] = pd.to_datetime(hour_data['timestamp']) 
hour_btc = scaling_data(hour_data)

In [ ]:
# Setting the sequence length for LSTM model, creating sequences from the hourly data, and applying the LSTM model to this data.
sequence_length = 20
X, y = create_sequences(hour_btc, sequence_length) # Prediction based on 20 hours 
y_train, y_test, predicted_values = lstm_model(X, y)
visualize_RNN_prediction(y_train, y_test, predicted_values)

In [ ]:
# Implementing a recursive prediction method.
t = 5
prediction = recursive_prediction(X, y, t)

In [ ]:
prediction = inverse_scalling(prediction, hour_data)

In [ ]:
# Plotting the results of the recursive prediction.
# The red dashed line at 'sequence_length' indicates the point in the time series where the recursive prediction begins.
plt.plot(prediction)
plt.title('Recursive prediction of the close price of BTC ')
plt.axvline(x = sequence_length, color = 'r', linestyle = '--')
plt.xlabel('time')
plt.ylabel('Price')
plt.legend('')
plt.show()